In [16]:
import tensorflow as tf  
import numpy as np  
from tensorflow.examples.tutorials.mnist import input_data
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

tf.reset_default_graph()
with tf.Graph().as_default() as g:
    # 构建层
    def add_layer(inputs, input_size, output_size, activation_function):
        with tf.name_scope('Layer'):
            with tf.name_scope('Weights'):
                Weights = tf.Variable(tf.random_normal([input_size, output_size]))
                tf.summary.histogram('Weights', Weights)
            with tf.name_scope('Bias'):
                bias = tf.Variable(tf.zeros([1, output_size]) + 0.1)
                tf.summary.histogram('Bias', bias)
            wx_b = tf.add(tf.matmul(inputs, Weights), bias)
            if activation_function != None:
                output = activation_function(wx_b)
            else:
                output = wx_b
            tf.summary.histogram('Output', output)
        return output

    # 定义准确率计算
    def computer_accuracy(v_x, v_y):
        global prediction
        y_pre = sess.run(prediction, feed_dict={xs: v_x})
        correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
        result = sess.run(accuracy, feed_dict={xs: v_x, ys: v_y})
        return result


    # 搭建网络 
    xs = tf.placeholder(tf.float32, [None, 784]) # 28*28
    ys = tf.placeholder(tf.float32, [None, 10])

    prediction = add_layer(xs, 784, 10, tf.nn.softmax)

    # 分类问题用交叉熵
    with tf.name_scope('CrossEntrphy'):
        cross_entrypy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))
        tf.summary.scalar('CrossEntrpyh', cross_entrypy)

    train = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entrypy)

    init = tf.global_variables_initializer()


    with tf.Session() as sess:        
        sess.run(init)
        writer = tf.summary.FileWriter('./logs', sess.graph)  
        mergerd = tf.summary.merge_all()         
        for i in range(1000):
            batch_x, batch_y = mnist.train.next_batch(100)
            sess.run(train, feed_dict={xs: batch_x, ys: batch_y})
            if i % 50 == 0:
                print(computer_accuracy(mnist.test.images, mnist.test.labels))
                rs = sess.run(mergerd, feed_dict={xs: batch_x, ys: batch_y})
                writer.add_summary(rs, i)


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
0.1404
0.639
0.7256
0.7758
0.8013
0.8107
0.8264
0.8376
0.8437
0.8497
0.8532
0.8549
0.8599
0.858
0.8641
0.8648
0.8663
0.8688
0.8719
0.8729
